# Notebook interactivo de EDA geoestadístico (datos reales)

Este notebook está diseñado para Google Colab y permite realizar un EDA geoestadístico interactivo con datos reales de minería.


In [ ]:

# ================================
# 0) SETUP
# ================================
import os
import json
import math
import shutil
from datetime import datetime

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import matplotlib

try:
    import seaborn as sns
    _HAS_SEABORN = True
except Exception:
    _HAS_SEABORN = False

try:
    import plotly.express as px
    _HAS_PLOTLY = True
except Exception:
    _HAS_PLOTLY = False

try:
    from scipy import stats
    _HAS_SCIPY = True
except Exception:
    _HAS_SCIPY = False

try:
    import ipywidgets as widgets
    from IPython.display import display, clear_output
except Exception as exc:
    raise ImportError("ipywidgets es necesario para este notebook interactivo.") from exc

# Crear carpeta outputs/eda/<timestamp>
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
output_dir = os.path.join("outputs", "eda", timestamp)
os.makedirs(output_dir, exist_ok=True)

print(f"✅ Notebook iniciado. Carpeta de salida: {output_dir}")


In [ ]:

# ================================
# Helpers
# ================================
def safe_sample(df, n, seed=42):
    if df is None:
        return df
    if len(df) <= n:
        return df
    return df.sample(n=n, random_state=seed)

def describe_stats(series):
    series = series.dropna()
    if series.empty:
        return {}
    stats_dict = {
        "mean": series.mean(),
        "median": series.median(),
        "min": series.min(),
        "max": series.max(),
        "range": series.max() - series.min(),
        "var": series.var(ddof=1),
        "std": series.std(ddof=1),
        "cv": series.std(ddof=1) / series.mean() if series.mean() != 0 else np.nan,
        "skew": series.skew(),
        "kurtosis": series.kurt(),
        "p5": series.quantile(0.05),
        "p10": series.quantile(0.10),
        "p25": series.quantile(0.25),
        "p50": series.quantile(0.50),
        "p75": series.quantile(0.75),
        "p90": series.quantile(0.90),
        "p95": series.quantile(0.95),
    }
    return stats_dict

def compute_outlier_mask(series, method, params):
    series = series.dropna()
    if series.empty:
        return pd.Series([False] * len(series), index=series.index)

    if method == "IQR":
        factor = params.get("iqr_factor", 1.5)
        q1 = series.quantile(0.25)
        q3 = series.quantile(0.75)
        iqr = q3 - q1
        lower = q1 - factor * iqr
        upper = q3 + factor * iqr
        return (series < lower) | (series > upper)

    if method == "Percentiles":
        low = params.get("p_low", 1)
        high = params.get("p_high", 99)
        lower = series.quantile(low / 100)
        upper = series.quantile(high / 100)
        return (series < lower) | (series > upper)

    if method == "Z-score":
        z = params.get("z", 3.0)
        zscores = (series - series.mean()) / series.std(ddof=1)
        return zscores.abs() > z

    if method == "Z-score robusto (MAD)":
        z = params.get("z", 3.5)
        median = series.median()
        mad = (series - median).abs().median()
        if mad == 0:
            return pd.Series([False] * len(series), index=series.index)
        robust_z = 0.6745 * (series - median) / mad
        return robust_z.abs() > z

    return pd.Series([False] * len(series), index=series.index)

def safe_hist(series, bins=30, log=False, ax=None, title="Histograma"):
    if ax is None:
        fig, ax = plt.subplots(figsize=(6, 4))
    else:
        fig = ax.figure
    data = series.dropna()
    if log:
        data = data[data > 0]
        ax.set_xscale("log")
    ax.hist(data, bins=bins, color="#3b82f6", alpha=0.7)
    ax.set_title(title)
    ax.set_ylabel("Frecuencia")
    return fig

def try_plotly_scatter_3d(df, x_col, y_col, z_col, color_col):
    if not _HAS_PLOTLY:
        return None
    try:
        fig = px.scatter_3d(df, x=x_col, y=y_col, z=z_col, color=color_col)
        return fig
    except Exception:
        return None

def save_matplotlib_figure(fig, name):
    if fig is None:
        return None
    path = os.path.join(output_dir, f"{name}.png")
    fig.savefig(path, bbox_inches="tight", dpi=150)
    return path

def export_zip(output_dir):
    zip_path = shutil.make_archive(output_dir, "zip", output_dir)
    return zip_path


In [ ]:

# ================================
# Estado global
# ================================
state = {
    "raw_bytes": None,
    "raw_name": None,
    "df_raw": None,
    "df_work": None,
    "df_clean": None,
    "figures": {},
    "config": {},
}


## 1) Carga y selección de datos (Interactivo)

In [ ]:

# Widgets de carga y selección
# Ejecutá esta celda (Shift+Enter) antes de usar Subir CSV. Si reiniciaste el runtime o abriste otra pestaña, volvé a ejecutarla.
import io

upload_button = widgets.Button(description="Subir CSV", button_style="primary")
separator_dropdown = widgets.Dropdown(options=["auto", ",", ";", "	"], value="auto", description="Separador")
decimal_dropdown = widgets.Dropdown(options=[".", ","], value=".", description="Decimal")

x_dropdown = widgets.Dropdown(options=[], description="X")
y_dropdown = widgets.Dropdown(options=[], description="Y")
z_dropdown = widgets.Dropdown(options=[], description="Z")
grade_dropdown = widgets.Dropdown(options=[], description="Grade")
domain_dropdown = widgets.Dropdown(options=["(ninguno)"], description="Domain")
litho_dropdown = widgets.Dropdown(options=["(ninguno)"], description="Lithology")

dropna_checkbox = widgets.Checkbox(value=True, description="Eliminar NaN automáticamente")
apply_button = widgets.Button(description="Aplicar selección", button_style="success")

load_output = widgets.Output()
selection_output = widgets.Output()

def _load_csv_from_bytes(raw_bytes, sep, decimal, encoding="utf-8"):
    buffer = io.BytesIO(raw_bytes)
    if sep == "auto":
        df = pd.read_csv(buffer, sep=None, engine="python", decimal=decimal, encoding=encoding)
    else:
        df = pd.read_csv(buffer, sep=sep, engine="python", decimal=decimal, encoding=encoding)
    return df

def handle_upload(_):
    with load_output:
        clear_output()
        try:
            from google.colab import files
            uploaded = files.upload()
        except Exception:
            print("El widget de upload solo funciona en Colab. Si estás en Colab, ejecutá esta celda nuevamente.")
            return

        if not uploaded:
            print("No se seleccionó ningún archivo.")
            return

        fname = next(iter(uploaded))
        raw = uploaded[fname]

        state["raw_bytes"] = raw
        state["raw_name"] = fname
        print(f"Archivo cargado: {fname}")

        # Validación temprana del CSV
        try:
            df_preview = _load_csv_from_bytes(raw, separator_dropdown.value, decimal_dropdown.value)
        except Exception as exc:
            print(f"Error leyendo CSV: {exc}")
            return

        print(f"Shape: {df_preview.shape}")
        print("Tipos de datos:")
        display(df_preview.dtypes)
        display(df_preview.head())

        if df_preview.shape[1] == 1:
            print("Parece que el separador no es correcto. Probá ';' y decimal ','")

upload_button.on_click(handle_upload)

def update_column_options(df):
    cols = list(df.columns)
    for widget in [x_dropdown, y_dropdown, z_dropdown, grade_dropdown]:
        widget.options = cols
    domain_dropdown.options = ["(ninguno)"] + cols
    litho_dropdown.options = ["(ninguno)"] + cols

def apply_selection(_):
    with selection_output:
        clear_output()
        if state["raw_bytes"] is None:
            print("Primero carga un CSV.")
            return

        try:
            df = _load_csv_from_bytes(state["raw_bytes"], separator_dropdown.value, decimal_dropdown.value)
        except Exception as exc:
            print(f"Error leyendo CSV: {exc}")
            return

        state["df_raw"] = df
        update_column_options(df)

        required_cols = [x_dropdown.value, y_dropdown.value, z_dropdown.value, grade_dropdown.value]
        missing = [col for col in required_cols if col not in df.columns]
        if missing:
            print("Selecciona columnas válidas antes de aplicar.")
            return

        selected_cols = required_cols[:]
        if domain_dropdown.value != "(ninguno)":
            selected_cols.append(domain_dropdown.value)
        if litho_dropdown.value != "(ninguno)":
            selected_cols.append(litho_dropdown.value)

        df_selected = df[selected_cols].copy()
        original_shape = df_selected.shape

        nan_percent = df_selected.isna().mean() * 100
        dup_count = df_selected.duplicated().sum()

        if dropna_checkbox.value:
            df_selected = df_selected.dropna()

        state["df_work"] = df_selected
        state["df_clean"] = None

        print(f"Shape original: {original_shape}")
        print(f"Shape filtrado: {df_selected.shape}")
        print("% NaN por columna:")
        display(nan_percent.to_frame("% NaN"))
        print(f"Duplicados detectados: {dup_count}")

        refresh_stats()
        refresh_spatial_widgets()
        refresh_swath_widgets()
        refresh_univariate_plots()

apply_button.on_click(apply_selection)

display(widgets.VBox([
    widgets.HTML("<b>Ejecutá esta celda (Shift+Enter) antes de usar Subir CSV. Si reiniciaste el runtime o abriste otra pestaña, volvé a ejecutarla.</b>"),
    widgets.HBox([upload_button, separator_dropdown, decimal_dropdown]),
    widgets.HBox([x_dropdown, y_dropdown, z_dropdown, grade_dropdown]),
    widgets.HBox([domain_dropdown, litho_dropdown]),
    dropna_checkbox,
    apply_button,
    load_output,
    selection_output,
]))


## 2) QA y rangos físicos (Interactivo)

In [ ]:

min_grade_input = widgets.FloatText(value=0.0, description="min_grade")
max_grade_input = widgets.FloatText(value=100.0, description="max_grade")
validate_button = widgets.Button(description="Validar rangos", button_style="warning")
qa_output = widgets.Output()

def validate_ranges(_):
    with qa_output:
        clear_output()
        df = state.get("df_work")
        if df is None or df.empty:
            print("Carga y selecciona datos primero.")
            return
        grade_col = grade_dropdown.value
        if grade_col not in df.columns:
            print("Selecciona la columna Grade.")
            return

        min_g = min_grade_input.value
        max_g = max_grade_input.value

        mask = (df[grade_col] < min_g) | (df[grade_col] > max_g)
        out_count = mask.sum()
        print(f"Valores fuera de rango: {out_count}")
        display(df.loc[mask].head(100))

validate_button.on_click(validate_ranges)

display(widgets.VBox([
    widgets.HBox([min_grade_input, max_grade_input, validate_button]),
    qa_output
]))


## 3) Outliers / Cutoff (Interactivo)

In [ ]:

outlier_method = widgets.Dropdown(
    options=["IQR", "Percentiles", "Z-score", "Z-score robusto (MAD)"],
    description="Método"
)
iqr_factor = widgets.FloatText(value=1.5, description="IQR factor")
p_low = widgets.IntSlider(value=1, min=0, max=10, step=1, description="P low")
p_high = widgets.IntSlider(value=99, min=90, max=100, step=1, description="P high")
z_score = widgets.FloatText(value=3.0, description="Z")

outlier_action = widgets.Dropdown(
    options=["Marcar outliers", "Eliminar outliers"],
    description="Acción"
)
apply_cutoff_button = widgets.Button(description="Aplicar cutoff", button_style="danger")

outlier_output = widgets.Output()

def refresh_outlier_params(*_):
    if outlier_method.value == "IQR":
        display_params = [iqr_factor]
    elif outlier_method.value == "Percentiles":
        display_params = [p_low, p_high]
    else:
        display_params = [z_score]
    params_box.children = display_params

def apply_cutoff(_):
    with outlier_output:
        clear_output()
        df = state.get("df_work")
        if df is None or df.empty:
            print("Carga y selecciona datos primero.")
            return
        grade_col = grade_dropdown.value
        if grade_col not in df.columns:
            print("Selecciona la columna Grade.")
            return

        params = {}
        if outlier_method.value == "IQR":
            params["iqr_factor"] = iqr_factor.value
        elif outlier_method.value == "Percentiles":
            params["p_low"] = p_low.value
            params["p_high"] = p_high.value
        else:
            params["z"] = z_score.value

        mask = compute_outlier_mask(df[grade_col], outlier_method.value, params)
        df_clean = df.copy()

        if outlier_action.value == "Marcar outliers":
            df_clean["outlier_flag"] = False
            df_clean.loc[mask.index, "outlier_flag"] = mask
        else:
            df_clean = df_clean.loc[~mask]

        state["df_clean"] = df_clean

        print(f"N original: {len(df)}")
        print(f"N limpio: {len(df_clean)}")

        fig, axes = plt.subplots(1, 2, figsize=(10, 4))
        safe_hist(df[grade_col], bins=30, ax=axes[0], title="Antes")
        safe_hist(df_clean[grade_col], bins=30, ax=axes[1], title="Después")
        plt.tight_layout()
        state["figures"]["outlier_hist"] = fig
        display(fig)

        fig2, axes2 = plt.subplots(1, 2, figsize=(10, 4))
        axes2[0].boxplot(df[grade_col].dropna(), vert=True)
        axes2[0].set_title("Antes")
        axes2[1].boxplot(df_clean[grade_col].dropna(), vert=True)
        axes2[1].set_title("Después")
        plt.tight_layout()
        state["figures"]["outlier_box"] = fig2
        display(fig2)

        refresh_stats()
        refresh_univariate_plots()
        refresh_spatial_plots()
        refresh_swath_plots()

outlier_method.observe(refresh_outlier_params, "value")
apply_cutoff_button.on_click(apply_cutoff)

params_box = widgets.HBox([iqr_factor])
refresh_outlier_params()

display(widgets.VBox([
    outlier_method,
    params_box,
    outlier_action,
    apply_cutoff_button,
    outlier_output
]))


## 4) Estadística descriptiva (Autoactualizada)

In [ ]:

stats_output = widgets.Output()

def build_stats_table(df, label):
    if df is None or df.empty:
        return None
    grade_col = grade_dropdown.value
    if grade_col not in df.columns:
        return None
    stats_dict = describe_stats(df[grade_col])
    df_stats = pd.DataFrame(stats_dict, index=[label]).T
    df_stats.columns = [label]
    return df_stats

def refresh_stats():
    with stats_output:
        clear_output()
        df_work = state.get("df_work")
        df_clean = state.get("df_clean")
        frames = []
        if df_work is not None:
            stats_work = build_stats_table(df_work, "original")
            if stats_work is not None:
                frames.append(stats_work)
        if df_clean is not None:
            stats_clean = build_stats_table(df_clean, "limpio")
            if stats_clean is not None:
                frames.append(stats_clean)

        if not frames:
            print("No hay datos para mostrar estadísticas.")
            return

        stats_table = pd.concat(frames, axis=1)
        display(stats_table)
        state["stats_table"] = stats_table

display(stats_output)


## 5) Visualización univariada (Interactivo)

In [ ]:

log_checkbox = widgets.Checkbox(value=False, description="Usar escala log")
bins_slider = widgets.IntSlider(value=30, min=5, max=100, step=1, description="Bins")
univariate_output = widgets.Output()

def refresh_univariate_plots(*_):
    with univariate_output:
        clear_output()
        df = state.get("df_clean") or state.get("df_work")
        if df is None or df.empty:
            print("No hay datos para graficar.")
            return
        grade_col = grade_dropdown.value
        if grade_col not in df.columns:
            print("Selecciona la columna Grade.")
            return

        data = df[grade_col]
        fig, axes = plt.subplots(2, 2, figsize=(12, 8))
        safe_hist(data, bins=bins_slider.value, log=log_checkbox.value, ax=axes[0, 0], title="Histograma")

        if _HAS_SEABORN:
            sns.kdeplot(data.dropna(), ax=axes[0, 1], fill=True)
            axes[0, 1].set_title("KDE")
        else:
            axes[0, 1].text(0.5, 0.5, "KDE no disponible", ha="center")

        if _HAS_SCIPY:
            stats.probplot(data.dropna(), dist="norm", plot=axes[1, 0])
            axes[1, 0].set_title("QQ plot normal")
        else:
            axes[1, 0].text(0.5, 0.5, "QQ normal no disponible", ha="center")

        if _HAS_SCIPY:
            data_log = data[data > 0].dropna()
            stats.probplot(np.log(data_log), dist="norm", plot=axes[1, 1])
            axes[1, 1].set_title("QQ plot lognormal")
        else:
            axes[1, 1].text(0.5, 0.5, "QQ lognormal no disponible", ha="center")

        plt.tight_layout()
        state["figures"]["univariate"] = fig
        display(fig)

log_checkbox.observe(refresh_univariate_plots, "value")
bins_slider.observe(refresh_univariate_plots, "value")

display(widgets.VBox([
    widgets.HBox([log_checkbox, bins_slider]),
    univariate_output
]))


## 6) Análisis espacial (Interactivo)

In [ ]:

subsample_slider = widgets.IntSlider(value=2000, min=100, max=20000, step=100, description="Subsample")
color_dropdown = widgets.Dropdown(options=["Grade"], description="Color")
view_dropdown = widgets.Dropdown(options=["Planta (XY)", "Sección X-Z", "Sección Y-Z"], description="Vista")
spatial_output = widgets.Output()

def refresh_spatial_widgets():
    df = state.get("df_work")
    if df is None:
        return
    options = [grade_dropdown.value]
    if domain_dropdown.value != "(ninguno)":
        options.append(domain_dropdown.value)
    if litho_dropdown.value != "(ninguno)":
        options.append(litho_dropdown.value)
    color_dropdown.options = options

def refresh_spatial_plots(*_):
    with spatial_output:
        clear_output()
        df = state.get("df_clean") or state.get("df_work")
        if df is None or df.empty:
            print("No hay datos para graficar.")
            return

        df_plot = safe_sample(df, subsample_slider.value)
        x_col, y_col, z_col = x_dropdown.value, y_dropdown.value, z_dropdown.value
        color_col = color_dropdown.value

        fig, ax = plt.subplots(figsize=(6, 5))
        if view_dropdown.value == "Planta (XY)":
            sc = ax.scatter(df_plot[x_col], df_plot[y_col], c=df_plot[color_col], cmap="viridis", s=10)
            ax.set_xlabel("X")
            ax.set_ylabel("Y")
        elif view_dropdown.value == "Sección X-Z":
            sc = ax.scatter(df_plot[x_col], df_plot[z_col], c=df_plot[color_col], cmap="viridis", s=10)
            ax.set_xlabel("X")
            ax.set_ylabel("Z")
        else:
            sc = ax.scatter(df_plot[y_col], df_plot[z_col], c=df_plot[color_col], cmap="viridis", s=10)
            ax.set_xlabel("Y")
            ax.set_ylabel("Z")

        plt.colorbar(sc, ax=ax, label=color_col)
        ax.set_title("Scatter espacial")
        plt.tight_layout()
        state["figures"]["spatial"] = fig
        display(fig)

subsample_slider.observe(refresh_spatial_plots, "value")
color_dropdown.observe(refresh_spatial_plots, "value")
view_dropdown.observe(refresh_spatial_plots, "value")

display(widgets.VBox([
    widgets.HBox([subsample_slider, color_dropdown, view_dropdown]),
    spatial_output
]))


## 7) Swath plots (Deriva) (Interactivo)

In [ ]:

swath_bins = widgets.IntSlider(value=10, min=5, max=50, step=1, description="Bins")
swath_direction = widgets.Dropdown(options=["X", "Y"], description="Dirección")
swath_filter = widgets.Dropdown(options=["(ninguno)"], description="Filtro")
swath_output = widgets.Output()

def refresh_swath_widgets():
    df = state.get("df_work")
    options = ["(ninguno)"]
    if df is None:
        swath_filter.options = options
        return
    if domain_dropdown.value != "(ninguno)":
        options.append(domain_dropdown.value)
    if litho_dropdown.value != "(ninguno)":
        options.append(litho_dropdown.value)
    swath_filter.options = options

def refresh_swath_plots(*_):
    with swath_output:
        clear_output()
        df = state.get("df_clean") or state.get("df_work")
        if df is None or df.empty:
            print("No hay datos para graficar.")
            return

        if swath_filter.value != "(ninguno)":
            df = df[df[swath_filter.value].notna()]

        coord_col = x_dropdown.value if swath_direction.value == "X" else y_dropdown.value
        grade_col = grade_dropdown.value
        if coord_col not in df.columns or grade_col not in df.columns:
            print("Columnas inválidas para swath.")
            return

        bins = swath_bins.value
        df = df[[coord_col, grade_col]].dropna()
        df["bin"] = pd.cut(df[coord_col], bins=bins)
        grouped = df.groupby("bin").agg(mean_grade=(grade_col, "mean"), count=(grade_col, "size"))

        fig, axes = plt.subplots(2, 1, figsize=(8, 6), sharex=True)
        axes[0].plot(grouped.index.astype(str), grouped["mean_grade"], marker="o")
        axes[0].set_ylabel("Media Grade")
        axes[0].set_title("Swath plot (media)")
        axes[1].bar(grouped.index.astype(str), grouped["count"], color="#94a3b8")
        axes[1].set_ylabel("Conteo")
        axes[1].set_xlabel("Bins")
        axes[1].tick_params(axis='x', rotation=90)
        plt.tight_layout()
        state["figures"]["swath"] = fig
        display(fig)

swath_bins.observe(refresh_swath_plots, "value")
swath_direction.observe(refresh_swath_plots, "value")
swath_filter.observe(refresh_swath_plots, "value")

display(widgets.VBox([
    widgets.HBox([swath_bins, swath_direction, swath_filter]),
    swath_output
]))


## 8) Visualización 3D (Opcional)

In [ ]:

subsample_3d = widgets.IntSlider(value=2000, min=100, max=20000, step=100, description="Subsample")
plot3d_output = widgets.Output()

def refresh_3d_plot(*_):
    with plot3d_output:
        clear_output()
        df = state.get("df_clean") or state.get("df_work")
        if df is None or df.empty:
            print("No hay datos para graficar.")
            return

        df_plot = safe_sample(df, subsample_3d.value)
        x_col, y_col, z_col = x_dropdown.value, y_dropdown.value, z_dropdown.value
        color_col = grade_dropdown.value

        fig_plotly = try_plotly_scatter_3d(df_plot, x_col, y_col, z_col, color_col)
        if fig_plotly is not None:
            fig_plotly.show()
            state["figures"]["plot3d_plotly"] = fig_plotly
            state["figures"]["plot3d"] = None
        else:
            fig = plt.figure(figsize=(6, 5))
            ax = fig.add_subplot(111, projection="3d")
            sc = ax.scatter(df_plot[x_col], df_plot[y_col], df_plot[z_col], c=df_plot[color_col], cmap="viridis", s=10)
            ax.set_xlabel("X")
            ax.set_ylabel("Y")
            ax.set_zlabel("Z")
            fig.colorbar(sc, ax=ax, label=color_col)
            plt.tight_layout()
            state["figures"]["plot3d"] = fig
            display(fig)

subsample_3d.observe(refresh_3d_plot, "value")

display(widgets.VBox([
    subsample_3d,
    plot3d_output
]))


## 9) Exportación de resultados

In [ ]:

export_button = widgets.Button(description="Exportar resultados", button_style="success")
export_output = widgets.Output()

def export_results(_):
    with export_output:
        clear_output()
        df_work = state.get("df_work")
        if df_work is None:
            print("No hay datos para exportar.")
            return

        df_clean = state.get("df_clean")
        df_work.to_csv(os.path.join(output_dir, "dataset_original.csv"), index=False)
        if df_clean is not None:
            df_clean.to_csv(os.path.join(output_dir, "dataset_clean.csv"), index=False)

        stats_table = state.get("stats_table")
        if stats_table is not None:
            stats_table.to_csv(os.path.join(output_dir, "stats_summary.csv"))

        # Guardar configuración
        config = {
            "separador": separator_dropdown.value,
            "decimal": decimal_dropdown.value,
            "x": x_dropdown.value,
            "y": y_dropdown.value,
            "z": z_dropdown.value,
            "grade": grade_dropdown.value,
            "domain": domain_dropdown.value,
            "lithology": litho_dropdown.value,
            "dropna": dropna_checkbox.value,
            "outlier_method": outlier_method.value,
            "outlier_action": outlier_action.value,
        }
        with open(os.path.join(output_dir, "run_config.json"), "w", encoding="utf-8") as f:
            json.dump(config, f, ensure_ascii=False, indent=2)

        # Guardar figuras matplotlib
        for name, fig in state["figures"].items():
            if name == "plot3d_plotly":
                continue
            save_matplotlib_figure(fig, name)

        # Si hay plotly, intentamos exportar
        fig_plotly = state["figures"].get("plot3d_plotly")
        if fig_plotly is not None:
            try:
                fig_plotly.write_image(os.path.join(output_dir, "plot3d_plotly.png"))
            except Exception:
                print("No se pudo exportar figura Plotly. Se omitirá o usar matplotlib en su lugar.")

        zip_path = export_zip(output_dir)

        print("✅ Resultados exportados:")
        print(f"- {output_dir}")
        print(f"- ZIP: {zip_path}")

        try:
            from google.colab import files
            files.download(zip_path)
        except Exception:
            print("Descarga automática no disponible en este entorno.")

export_button.on_click(export_results)

display(widgets.VBox([
    export_button,
    export_output
]))
